# Import Libraries

In [41]:
import os
import sys
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt

GFE_DIR = ''
sys.path.append(GFE_DIR)
import JKsKIRT as gskirt

from rur import uri, uhmi

In [2]:
SioverSil = 0.163 
# Silicate in NC --- assuming a fixed amorphous olivine composition(MgFeSiO4).
#                    assuming pyroxene with MgFeSi2O6 instead of olivine 
#                    typically increase the amount of dust mass released as silicates by 20%.

# Test

## load sim raw data

In [3]:
nout = 341

repo = '/storage7/NewCluster2/'
path_in_repo = 'snapshots'
mode = 'nc'

boxrad = 15*np.sqrt(3)

In [4]:
snap = uri.RamsesSnapshot(repo=repo, 
                          path_in_repo=path_in_repo, 
                          iout=nout, 
                          mode=mode, 
                          longint=False)

#boxtokpc = 1/snap.unit['kpc']#/snap.params['aexp']

[Output 00341] Age (Gyr) : 4.827 / 13.761, z = 1.33203 (a = 0.4288)


In [5]:
gals_tot, pid = uhmi.HaloMaker.load( snap, 
                                    galaxy=True,
                                    load_parts=True,
                                    double_precision=True)
gals_tot.sort(order='m')

In [6]:
mlow = 10
mupp = 13

gals = gals_tot[(10**mlow<=gals_tot['mvir'])*(gals_tot['mvir']<10**mupp)]
print(gals['id'])

[ 518  188    9  894  524  506  418  439  120  288  111  333   94  306
  334  425   58  264   67  562  367   99   46  270  646   53  341  697
  249  629  198  263  100  289   41  350  160  392  436 3794 4085   20
  446  116  133   75  237   76  577  512  443  401  134  243   30  269
  497   57  541  476   54   70  301   78   47  195  423 2976   25    2
  296  377 4387  328  107  340  235   72  317  387  262  143   37  138
 4362   87  139   79  654  432   44  124  208   81  596  101    6   40
   32   85 4869  155   11   92   66  106   97   29  311 3763  197  204
  245   52 3745   71    1   33 4384   84   89   98   50 4284   77  164
   36  324   64    7   80   14  167   13   31   88    4   22 3919  162
 3911   42    8  141   21   23   17 4190   27   12    3   19   18   15
   10    5]


In [24]:
idgal = 31

In [25]:
gal = gals[gals['id']==idgal]

In [26]:
snap.clear()
snap = uri.RamsesSnapshot(repo=repo, 
                          path_in_repo=path_in_repo, 
                          iout=nout, 
                          mode=mode)

boxtokpc = 1/snap.unit['kpc']
snap.set_box_halo(gal, 
                  use_halo_radius=False, 
                  radius=boxrad/boxtokpc)

star = snap.get_part(pname='star')

snap.clear()
snap = uri.RamsesSnapshot(repo=repo, 
                          path_in_repo=path_in_repo, 
                          iout=nout, 
                          mode=mode)
snap.set_box_halo(gal, 
                  use_halo_radius=False, 
                  radius=boxrad/boxtokpc)
snap.get_cell()
cell = snap.cell    

[clear] Clearing memory
	Unlink `rur_cell_nc_u1034_20240502_150303_851356`
[Output 00341] Age (Gyr) : 4.827 / 13.761, z = 1.33203 (a = 0.4288)
Reading 8 part files (1.400 GiB) in /storage7/NewCluster2/snapshots/output_00341... 
Allocating Memory...


/home/jangjk/rur/rur/uri.py:845: UserWarning: numpy.ndarray is passed instead of np.void in halo parameter. Assuming first row as input halo...
  warnings.warn(


Done (0.767 sec)



Done (4.425s).
Masking particles... 5818244 / 7102819 (0.8191)
Done (7.178s).
[clear] Clearing memory
	Unlink `rur_part_nc_u1034_20240502_152613_814360`
[Output 00341] Age (Gyr) : 4.827 / 13.761, z = 1.33203 (a = 0.4288)
Reading 8 AMR & hydro files (631.0 MiB) in /storage7/NewCluster2/snapshots/output_00341... 
Allocating Memory...
Done (1.342 sec)



Done (3.788s).
Masking cells... 1048395 / 2524958 (0.4152)
Done (0.208s).


In [27]:
mC_small = cell['d1']*cell['m','Msun']
mC_large = cell['d2']*cell['m','Msun']
mSil_small = cell['d3']*cell['m','Msun']/SioverSil
mSil_large = cell['d4']*cell['m','Msun']/SioverSil

In [28]:
print('Total small Graphite Mass = %s (log Msun)'%(np.round(np.log10(np.sum(mC_small)),2)))
print('Total large Graphite Mass = %s (log Msun)'%(np.round(np.log10(np.sum(mC_large)),2)))
print('Total small Silicate Mass = %s (log Msun)'%(np.round(np.log10(np.sum(mSil_small)),2)))
print('Total large Silicate Mass = %s (log Msun)'%(np.round(np.log10(np.sum(mSil_large)),2)))

Total small Graphite Mass = 7.33 (log Msun)
Total large Graphite Mass = 7.95 (log Msun)
Total small Silicate Mass = 7.1 (log Msun)
Total large Silicate Mass = 7.57 (log Msun)


## load param file

In [37]:
repo_par = '/home/jangjk/Project/JKsKIRT/example/'
fname_par = 'parameter_setting'

try:
    par = reload(sys.modules[fname_par])
except:
    pass

sys.path.append(repo_par)
par = __import__(fname_par)

In [38]:
par.skirt_dir = '/storage5/scratch/library/SKIRT/release/SKIRT/main/'
par.N_phot = 4e7

In [39]:
print('Total photon packet Number = 10^{%s}'%(np.round(np.log10(par.N_phot),2)),end='\n\n')
print('Redshift of the target = %s'%(par.z_red))
print('Distance to the target = %s (>0 only when z_red = 0)'%(par.inst_dist),end='\n\n')
print('Field-of-View (X-direction) = %s %s'%(par.fov_X,par.fov_unit))
print('Field-of-View (Y-direction) = %s %s'%(par.fov_Y,par.fov_unit),end='\n\n')
print('Using on-the-fly-dust info = %s'%(par.on_the_fly_dust),end='\n\n')
print('The file will be saved in %s/%s'%(par.repo,par.repo_output),end='\n\n')

Total photon packet Number = 10^{7.6}

Redshift of the target = 3
Distance to the target = 0 (>0 only when z_red = 0)

Field-of-View (X-direction) = 40 kpc
Field-of-View (Y-direction) = 40 kpc

Using on-the-fly-dust info = True

The file will be saved in /home/jangjk//result/



## execute SKIRT 

In [40]:
gskirt.utils.execute_SKIRT.make_INSKI(
                                   boxtokpc=boxtokpc,
                                   x_s=star['x'],y_s=star['y'],z_s=star['z'],
                                   vx_s=star['vx','km/s'],vy_s=star['vy','km/s'],vz_s=star['vz','km/s'],
                                   m_s=star['m','Msun'],m0_s=star['m0','Msun'],
                                   age_s=star['age','Gyr'],metal_s=star['metal'],
                                   x_c=cell['x'],y_c=cell['y'],z_c=cell['z'],lvl_c=cell['level'],
                                   vx_c=cell['vx','km/s'],vy_c=cell['vy','km/s'],vz_c=cell['vz','km/s'],
                                   m_c=cell['m','Msun'],
                                   T_c=cell['T','K'],metal_c=cell['metal'],
                                   param=par,
                                   pos_ctr=[gal['x'],gal['y'],gal['z']],
                                   vel_ctr=[gal['vx'],gal['vy'],gal['vz']],
                                   mC_small=mC_small,
                                   mC_large=mC_large,
                                   mSil_small=mSil_small,
                                   mSil_large=mSil_large,
                                  )


done -- basic setting
start -- calculating distance to the Nth-nearest neighbor (N=5)
done -- calculating distance to the Nth-nearest neighbor (N=5)
done -- writing old population
done -- writing gas cells
02/05/2024 15:42:21.983   Welcome to SKIRT v9.0 (git 2191a4e built on 23/03/2024 at 17:15:39)
02/05/2024 15:42:21.983   Running on gc7 for jangjk
02/05/2024 15:42:21.983   Constructing a simulation from ski file '../nh_test.ski'...
02/05/2024 15:42:28.781   Starting simulation nh_test using 40 threads and a single process...
02/05/2024 15:42:28.781   Starting setup...
02/05/2024 15:42:28.782     Panchromatic wavelength regime
02/05/2024 15:42:28.782     With transfer medium
02/05/2024 15:42:28.782     Redshift: 3
02/05/2024 15:42:28.782     Luminosity distance: 26109.7 Mpc
02/05/2024 15:42:28.782     Model and grid symmetry: 3D
02/05/2024 15:42:28.782     Photon life cycle: no explicit absorption; with forced scattering
02/05/2024 15:42:28.804   Starburst99SEDFamily opened stored tab

02/05/2024 15:44:23.098   Done reading
02/05/2024 15:44:23.098     Number of cells: 1048395
02/05/2024 15:44:23.139     Total original mass : 1.2619e+07 Msun
02/05/2024 15:44:23.139     Total effective mass: 1.2619e+07 Msun
02/05/2024 15:44:23.152   Constructing intermediate 20x20x20 grid for cells...
02/05/2024 15:44:23.374     Smallest number of cells per grid cell: 6
02/05/2024 15:44:23.374     Largest  number of cells per grid cell: 7446
02/05/2024 15:44:23.375     Average  number of cells per grid cell: 218.8
02/05/2024 15:44:23.377   ConfigurableDustMix opened stored table /storage5/scratch/library/SKIRT/resources/SKIRT9_Resources_Core/OpticalProps/DraineSilicateOpticalProps.stab
02/05/2024 15:44:23.596   ConfigurableDustMix allocated 52.9 KB of memory
02/05/2024 15:44:23.596   CellMedium reads cuboidal cells from text file /home/jangjk//gas_cell.txt...
02/05/2024 15:44:23.597     Column 1: box xmin (kpc) <-- x-min
02/05/2024 15:44:23.597     Column 2: box ymin (kpc) <-- y-min
02

02/05/2024 15:50:18.573   Launched primary emission photon packets: 37.8%
02/05/2024 15:50:23.581   Launched primary emission photon packets: 38.9%
02/05/2024 15:50:28.585   Launched primary emission photon packets: 40.0%
02/05/2024 15:50:33.592   Launched primary emission photon packets: 41.1%
02/05/2024 15:50:38.594   Launched primary emission photon packets: 42.3%
02/05/2024 15:50:43.621   Launched primary emission photon packets: 43.4%
02/05/2024 15:50:48.632   Launched primary emission photon packets: 44.5%
02/05/2024 15:50:53.640   Launched primary emission photon packets: 45.5%
02/05/2024 15:50:58.649   Launched primary emission photon packets: 46.3%
02/05/2024 15:51:03.651   Launched primary emission photon packets: 47.4%
02/05/2024 15:51:08.655   Launched primary emission photon packets: 48.5%
02/05/2024 15:51:13.668   Launched primary emission photon packets: 49.4%
02/05/2024 15:51:18.683   Launched primary emission photon packets: 50.4%
02/05/2024 15:51:23.703   Launched pri

02/05/2024 15:56:02.912   Available memory: 315 GB -- Peak memory usage: 2.36 GB (0.8%)
